In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import LMRt
import os
import statsmodels.api as sm                                                                                              
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
dirpath = '/home/fzhu/Apps/LMRt/LMRt/data'

db_proxies_filename = 'LMRdb_v1.1.0_Proxies.df.pckl'
db_metadata_filename = 'LMRdb_v1.1.0_Metadata.df.pckl'

job = LMRt.ReconJob()
db_proxies_filepath = os.path.join(dirpath, db_proxies_filename)
db_metadata_filepath = os.path.join(dirpath, db_metadata_filename)

job.cfg.proxies.proxy_frac = 1

job.load_proxies(db_proxies_filepath, db_metadata_filepath,
                 print_proxy_type_list=True, verbose=False)

pid=14640 >>> job.cfg created
pid=14640 >>> job.proxy_manager created

Proxy types
--------------
                                 Bivalve_d18O:    1
               Corals and Sclerosponges_Rates:    8
                Corals and Sclerosponges_SrCa:   25
                Corals and Sclerosponges_d18O:   60
                        Ice Cores_MeltFeature:    1
                               Ice Cores_d18O:   31
                                 Ice Cores_dD:    7
                              Lake Cores_Misc:    3
                             Lake Cores_Varve:    6
                       Tree Rings_WidthPages2:  347
                       Tree Rings_WoodDensity:   59
                                        TOTAL:  548


In [3]:
seasons_list = [
    [1,2,3,4,5,6,7,8,9,10,11,12],
    [6,7,8],
    [3,4,5,6,7,8],
    [6,7,8,9,10,11],
    [-12,1,2],
    [-9,-10,-11,-12,1,2],
    [-12,1,2,3,4,5],
] 
for pobj in job.proxy_manager.all_proxies:
    seasons_list.append(pobj.seasonality)
    
seasons_set = set(map(tuple, seasons_list))
avgMonths_list = list(map(list, seasons_set))
print(avgMonths_list)

[[-12, 1, 2, 3], [-5, -6, -7, -8, -9, -10, -11, -12, 1, 2, 3, 4], [-9, -10, -11, -12, 1, 2], [4, 5, 6, 7, 8, 9], [6, 7], [9, 10, 11], [-12, 1, 2], [-10, -11, -12, 1, 2, 3, 4], [6, 7, 8, 9, 10, 11], [7], [-12, -11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [7, 8, 9], [6, 7, 8], [3, 4, 5, 6, 7, 8, 9, 10], [4, 5, 6], [3, 4, 5, 6, 7, 8], [4], [-12, 1, 2, 3, 4, 5], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [-9, -10, -11, -12, 1, 2, 3, 4], [-12, -11, -10, -9, 1, 2, 3, 4, 5, 6, 7, 8], [6], [3, 4, 5], [-12, -11, -10, -9, -8, -7, 1, 2, 3, 4, 5, 6]]


In [4]:
# GISTEMP & GPCC
ana_dirpath = '/home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/analyses'

inst_field, inst_time, inst_lat, inst_lon = LMRt.utils.load_inst_analyses(
        {
            'GISTEMP': os.path.join(ana_dirpath, 'GISTEMP', 'gistemp1200_ERSSTv4.nc'),
            'GPCC': os.path.join(ana_dirpath, 'GPCC', 'GPCC_precip.mon.flux.1x1.v6.nc'),
        },
        var='field',
        verif_yrs=None,
        ref_period=[1951, 1980], outfreq='monthly',
)

for dataset_name in inst_time.keys():
    print(f'Processing {dataset_name} ...')
    LMRt.utils.calc_seasonal_avg(
        inst_field[dataset_name], inst_time[dataset_name],
        lat=inst_lat[dataset_name], lon=inst_lon[dataset_name],
        save_path=f'./data/seasonal_avg/seasonal_{dataset_name}.pkl',
        seasonality=avgMonths_list,
        make_yr_mm_nan=True,
        verbose=True,
    )

Loading GISTEMP: /home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/analyses/GISTEMP/gistemp1200_ERSSTv4.nc ...
Loading GPCC: /home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/analyses/GPCC/GPCC_precip.mon.flux.1x1.v6.nc ...
Processing GISTEMP ...
>>> Processing [-12, 1, 2, 3]
>>> Processing [-5, -6, -7, -8, -9, -10, -11, -12, 1, 2, 3, 4]
>>> Processing [-9, -10, -11, -12, 1, 2]
>>> Processing [4, 5, 6, 7, 8, 9]
>>> Processing [6, 7]
>>> Processing [9, 10, 11]
>>> Processing [-12, 1, 2]
>>> Processing [-10, -11, -12, 1, 2, 3, 4]
>>> Processing [6, 7, 8, 9, 10, 11]
>>> Processing [7]
>>> Processing [-12, -11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
>>> Processing [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
>>> Processing [7, 8, 9]
>>> Processing [6, 7, 8]
>>> Processing [3, 4, 5, 6, 7, 8, 9, 10]
>>> Processing [4, 5, 6]
>>> Processing [3, 4, 5, 6, 7, 8]
>>> Processing [4]
>>> Processing [-12, 1, 2, 3, 4, 5]
>>> Processing [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
>>> P

In [8]:
# iCESM
prior_dirpath = '/home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/model/icesm_last_millennium_historical'
tas_filename = 'tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc'
pr_filename = 'pr_sfc_Amon_iCESM_past1000historical_085001-200512.nc'
prior_filesdict = {
    'tas': os.path.join(prior_dirpath, tas_filename),
    'pr': os.path.join(prior_dirpath, pr_filename),
}

lat_model, lon_model, time_model, prior_vars = LMRt.utils.get_env_vars(prior_filesdict, calc_anomaly=True)

for var_name, prior_var in prior_vars.items():
    print(f'Processing {var_name} ...')
    LMRt.utils.calc_seasonal_avg(
        prior_var, time_model,
        lat=lat_model, lon=lon_model,
        save_path=f'./data/seasonal_avg/seasonal_iCESM_{var_name}.pkl',
        verbose=True, make_yr_mm_nan=False, seasonality=avgMonths_list,
    )

Processing tas ...
>>> Processing [-12, 1, 2, 3]
>>> Processing [-5, -6, -7, -8, -9, -10, -11, -12, 1, 2, 3, 4]
>>> Processing [-9, -10, -11, -12, 1, 2]
>>> Processing [4, 5, 6, 7, 8, 9]
>>> Processing [6, 7]
>>> Processing [9, 10, 11]
>>> Processing [-12, 1, 2]
>>> Processing [-10, -11, -12, 1, 2, 3, 4]
>>> Processing [6, 7, 8, 9, 10, 11]
>>> Processing [7]
>>> Processing [-12, -11, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
>>> Processing [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
>>> Processing [7, 8, 9]
>>> Processing [6, 7, 8]
>>> Processing [3, 4, 5, 6, 7, 8, 9, 10]
>>> Processing [4, 5, 6]
>>> Processing [3, 4, 5, 6, 7, 8]
>>> Processing [4]
>>> Processing [-12, 1, 2, 3, 4, 5]
>>> Processing [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
>>> Processing [-9, -10, -11, -12, 1, 2, 3, 4]
>>> Processing [-12, -11, -10, -9, 1, 2, 3, 4, 5, 6, 7, 8]
>>> Processing [6]
>>> Processing [3, 4, 5]
>>> Processing [-12, -11, -10, -9, -8, -7, 1, 2, 3, 4, 5, 6]
Processing pr ...
>>> Processing [-12, 1, 2, 3]
>>> Pr

In [5]:
ptypes = [item[0] for item in job.proxy_manager.proxy_type_list]
print(ptypes)

precalib_savepath = './data/precalib/calib_linear_GISTEMP.pkl'
job.build_precalib_files(
    ptypes, 'linear', precalib_savepath, nproc=1, verbose=False,
    precalc_avg_pathdict={
        'T': './data/seasonal_avg/seasonal_GISTEMP.pkl',
    }
)

['Bivalve_d18O', 'Corals and Sclerosponges_Rates', 'Corals and Sclerosponges_SrCa', 'Corals and Sclerosponges_d18O', 'Ice Cores_MeltFeature', 'Ice Cores_d18O', 'Ice Cores_dD', 'Lake Cores_Misc', 'Lake Cores_Varve', 'Tree Rings_WidthPages2', 'Tree Rings_WoodDensity']
pid=14640 >>> 1/548: PAGES2kv2_NAm-nv512_NAm_049:trsgi
pid=14640 >>> 2/548: PAGES2kv2_Asia-QAMDJT_Asi_221:trsgi
pid=14640 >>> 3/548: PAGES2kv2_Asia-ONONLS_Asi_153:trsgi
pid=14640 >>> 4/548: PAGES2kv2_Asia-GOUQIN_Asi_044:trsgi
pid=14640 >>> 5/548: PAGES2kv2_Asia-SHIYAT_Asi_130:trsgi
pid=14640 >>> 6/548: PAGES2kv2_NAm-cana352_NAm_030:trsgi
pid=14640 >>> 7/548: PAGES2kv2_NAm-mt119_NAm_046:trsgi
pid=14640 >>> 8/548: PAGES2kv2_NAm-mt108_NAm_178:trsgi
pid=14640 >>> 9/548: PAGES2kv2_Asia-BIARLS_Asi_142:trsgi
pid=14640 >>> 10/548: PAGES2kv2_Asia-HORBLS_Asi_146:trsgi
pid=14640 >>> 11/548: PAGES2kv2_SAm-CAN11.Neukom.2011_SAm_006:trsgi
pid=14640 >>> 12/548: PAGES2kv2_Asia-BULGLS_Asi_143:trsgi
pid=14640 >>> 13/548: PAGES2kv2_Aus-Stewar

In [6]:
precalib_savepath = './data/precalib/calib_bilinear_GISTEMP_GPCC.pkl'
job.build_precalib_files(
    ptypes, 'bilinear', precalib_savepath, nproc=1, verbose=False,
    precalc_avg_pathdict={
        'T': './data/seasonal_avg/seasonal_GISTEMP.pkl',
        'M': './data/seasonal_avg/seasonal_GPCC.pkl',
    }
)

pid=14640 >>> 1/548: PAGES2kv2_NAm-nv512_NAm_049:trsgi
pid=14640 >>> 2/548: PAGES2kv2_Asia-QAMDJT_Asi_221:trsgi
pid=14640 >>> 3/548: PAGES2kv2_Asia-ONONLS_Asi_153:trsgi
pid=14640 >>> 4/548: PAGES2kv2_Asia-GOUQIN_Asi_044:trsgi
pid=14640 >>> 5/548: PAGES2kv2_Asia-SHIYAT_Asi_130:trsgi
pid=14640 >>> 6/548: PAGES2kv2_NAm-cana352_NAm_030:trsgi
pid=14640 >>> 7/548: PAGES2kv2_NAm-mt119_NAm_046:trsgi
pid=14640 >>> 8/548: PAGES2kv2_NAm-mt108_NAm_178:trsgi
pid=14640 >>> 9/548: PAGES2kv2_Asia-BIARLS_Asi_142:trsgi
pid=14640 >>> 10/548: PAGES2kv2_Asia-HORBLS_Asi_146:trsgi
pid=14640 >>> 11/548: PAGES2kv2_SAm-CAN11.Neukom.2011_SAm_006:trsgi
pid=14640 >>> 12/548: PAGES2kv2_Asia-BULGLS_Asi_143:trsgi
pid=14640 >>> 13/548: PAGES2kv2_Aus-StewartIslan.DArrigo.1995_Aus_030:trsgi
pid=14640 >>> 14/548: PAGES2kv2_NAm-cana437_NAm_156:trsgi
pid=14640 >>> 15/548: PAGES2kv2_Asia-KOKKIY_Asi_137:trsgi
pid=14640 >>> 16/548: PAGES2kv2_NAm-ca534_NAm_011:trsgi
pid=14640 >>> 17/548: PAGES2kv2_NAm-mt116_NAm_045:trsgi
pid=1

In [9]:
%%time

prior_dirpath = '/home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/model/icesm_last_millennium_historical'
tas_filename = 'tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc'
prior_filesdict = {
    'tas': os.path.join(prior_dirpath, tas_filename),
}

ye_savepath = './data/Ye/Ye_iCESM_850_2005_linear_GISTEMP.npz'

job.build_ye_files(
    ptypes, 'linear',
    None, ye_savepath,
    precalib_filesdict={
        'linear': './data/precalib/calib_linear_GISTEMP.pkl'
    },
    precalc_avg_pathdict={
        'T': './data/seasonal_avg/seasonal_iCESM_tas.pkl',
    },
    nproc=1,
    verbose=False,
)

pid=14640 >>> 1/548: PAGES2kv2_NAm-nv512_NAm_049:trsgi
pid=14640 >>> 2/548: PAGES2kv2_Asia-QAMDJT_Asi_221:trsgi
pid=14640 >>> 3/548: PAGES2kv2_Asia-ONONLS_Asi_153:trsgi
pid=14640 >>> 4/548: PAGES2kv2_Asia-GOUQIN_Asi_044:trsgi
pid=14640 >>> 5/548: PAGES2kv2_Asia-SHIYAT_Asi_130:trsgi
pid=14640 >>> 6/548: PAGES2kv2_NAm-cana352_NAm_030:trsgi
pid=14640 >>> 7/548: PAGES2kv2_NAm-mt119_NAm_046:trsgi
pid=14640 >>> 8/548: PAGES2kv2_NAm-mt108_NAm_178:trsgi
pid=14640 >>> 9/548: PAGES2kv2_Asia-BIARLS_Asi_142:trsgi
pid=14640 >>> 10/548: PAGES2kv2_Asia-HORBLS_Asi_146:trsgi
pid=14640 >>> 11/548: PAGES2kv2_SAm-CAN11.Neukom.2011_SAm_006:trsgi
pid=14640 >>> 12/548: PAGES2kv2_Asia-BULGLS_Asi_143:trsgi
pid=14640 >>> 13/548: PAGES2kv2_Aus-StewartIslan.DArrigo.1995_Aus_030:trsgi
pid=14640 >>> 14/548: PAGES2kv2_NAm-cana437_NAm_156:trsgi
pid=14640 >>> 15/548: PAGES2kv2_Asia-KOKKIY_Asi_137:trsgi
pid=14640 >>> 16/548: PAGES2kv2_NAm-ca534_NAm_011:trsgi
pid=14640 >>> 17/548: PAGES2kv2_NAm-mt116_NAm_045:trsgi
pid=1

In [10]:
%%time

prior_dirpath = '/home/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/model/icesm_last_millennium_historical'
tas_filename = 'tas_sfc_Amon_iCESM_past1000historical_085001-200512.nc'
prior_filesdict = {
    'tas': os.path.join(prior_dirpath, tas_filename),
}

ye_savepath = './data/Ye/Ye_iCESM_850_2005_bilinear_GISTEMP_GPCC.npz'

job.build_ye_files(
    ptypes, 'bilinear',
    None, ye_savepath,
    precalib_filesdict={
        'bilinear': './data/precalib/calib_bilinear_GISTEMP_GPCC.pkl'
    },
    precalc_avg_pathdict={
        'T': './data/seasonal_avg/seasonal_iCESM_tas.pkl',
        'M': './data/seasonal_avg/seasonal_iCESM_pr.pkl',
    },
    nproc=1,
    verbose=False,
)

pid=14640 >>> 1/548: PAGES2kv2_NAm-nv512_NAm_049:trsgi
pid=14640 >>> 2/548: PAGES2kv2_Asia-QAMDJT_Asi_221:trsgi
pid=14640 >>> 3/548: PAGES2kv2_Asia-ONONLS_Asi_153:trsgi
pid=14640 >>> 4/548: PAGES2kv2_Asia-GOUQIN_Asi_044:trsgi
pid=14640 >>> 5/548: PAGES2kv2_Asia-SHIYAT_Asi_130:trsgi
pid=14640 >>> 6/548: PAGES2kv2_NAm-cana352_NAm_030:trsgi
pid=14640 >>> 7/548: PAGES2kv2_NAm-mt119_NAm_046:trsgi
pid=14640 >>> 8/548: PAGES2kv2_NAm-mt108_NAm_178:trsgi
pid=14640 >>> 9/548: PAGES2kv2_Asia-BIARLS_Asi_142:trsgi
pid=14640 >>> 10/548: PAGES2kv2_Asia-HORBLS_Asi_146:trsgi
pid=14640 >>> 11/548: PAGES2kv2_SAm-CAN11.Neukom.2011_SAm_006:trsgi
pid=14640 >>> 12/548: PAGES2kv2_Asia-BULGLS_Asi_143:trsgi
pid=14640 >>> 13/548: PAGES2kv2_Aus-StewartIslan.DArrigo.1995_Aus_030:trsgi
pid=14640 >>> 14/548: PAGES2kv2_NAm-cana437_NAm_156:trsgi
pid=14640 >>> 15/548: PAGES2kv2_Asia-KOKKIY_Asi_137:trsgi
pid=14640 >>> 16/548: PAGES2kv2_NAm-ca534_NAm_011:trsgi
pid=14640 >>> 17/548: PAGES2kv2_NAm-mt116_NAm_045:trsgi
pid=1